In [1]:
######### tf.group
import tensorflow as tf

sess = tf.InteractiveSession()
var = tf.Variable(1.0,dtype=tf.float32)
op1 = tf.assign_add(var, 1.0)
op2 = tf.assign_add(var, 100.0)
op3 = tf.no_op()

tf.global_variables_initializer().run()
ops = tf.group(op1,op2,op3)
sess.run(ops)
print(var.eval())

102.0


In [2]:
###########  tf.trainable_variables() tf.global_variables
import tensorflow as tf;    
import numpy as np;      
  
v = tf.Variable(tf.constant(0.0, shape=[1], dtype=tf.float32), name='v')  
v1 = tf.Variable(tf.constant(5, shape=[1], dtype=tf.float32), name='v1')  
  
global_step = tf.Variable(tf.constant(5, shape=[1], dtype=tf.float32), name='global_step', trainable=False)  
  
for ele1 in tf.trainable_variables():  
    print (ele1.name)
print()
for ele2 in tf.global_variables():  
    print (ele2.name)  

v:0
v1:0
v_1:0
v1_1:0

v:0
v1:0
global_step:0
v_1:0
v1_1:0
global_step_1:0


In [1]:
################# tf.train.ExponentialMovingAverage
import tensorflow as tf
import numpy as np

sess = tf.InteractiveSession()

v = tf.Variable(tf.constant(4.0, shape=[1], dtype=tf.float32), name='v')

ema = tf.train.ExponentialMovingAverage(0.99)
maintain_average_op = ema.apply(tf.global_variables())
a = tf.global_variables()
ma = ema.average(v)

tf.global_variables_initializer().run()
for ele in a:
    print(ele.name)
    
print()
print(v.eval())
print(ma.eval())

v:0
v/ExponentialMovingAverage:0

[ 4.]


In [1]:
################# tf.train.ExponentialMovingAverage
import tensorflow as tf
w = tf.Variable(1.0)
ema = tf.train.ExponentialMovingAverage(0.9)
update = tf.assign_add(w, 1.0)

with tf.control_dependencies([update]):
    #返回一个op,这个op用来更新moving_average
    ema_op = ema.apply([w])#这句和下面那句不能调换顺序
# 此op用来返回当前的moving_average
ema_val = ema.average(w)#参数不能是list，有点蛋疼

with tf.Session() as sess:
    a = tf.global_variables()
    tf.global_variables_initializer().run()
    for i in range(3):
        sess.run(ema_op)
        print(sess.run(ema_val))
    print()
    for ele in a:
        print(ele.name)
        print(ele.eval())
# 创建一个时间序列 1 2 3 4
#输出：
#1.1      =0.9*1 + 0.1*2
#1.29     =0.9*1.1+0.1*3
#1.561    =0.9*1.29+0.1*4

1.1
1.29
1.561

Variable:0
4.0
Variable/ExponentialMovingAverage:0
1.561


In [1]:
##############  tf.nn.conv3d
import tensorflow as tf
import numpy as np

sess = tf.InteractiveSession()
input = np.arange(1204224).reshape([2,16,112,112,3])
filter = np.arange(5184).reshape([3,3,3,3,64])

padding = 'SAME'
strides = [1,1,1,1,1]
res = tf.nn.conv3d(input,filter,strides,padding)

print(res.get_shape())

pool = tf.nn.max_pool3d(res,ksize = [1,1,2,2,1],strides = [1,1,2,2,1],padding='SAME')
print(pool.get_shape())

w2 = np.arange(221184).reshape([3,3,3,64,128])
res2 = tf.nn.conv3d(pool,w2,strides,padding)
print(res2.get_shape())

(2, 16, 112, 112, 64)
(2, 16, 56, 56, 64)
(2, 16, 56, 56, 128)


In [2]:
print(2*2*7*7*512)

100352


In [6]:
###############  tf.nn.max_pool3d
import tensorflow as tf
import numpy as np

x = np.arange(100352).reshape(2,2,7,7,512)
pool = tf.nn.max_pool3d(x,ksize = [1,2,2,2,1],strides = [1,2,2,2,1],padding='SAME')
print(pool.get_shape())
pool = tf.transpose(pool, perm=[0,1,4,2,3])
print(pool.get_shape())
print(pool.get_shape().as_list())

(2, 1, 4, 4, 512)
(2, 1, 512, 4, 4)
[2, 1, 512, 4, 4]


In [1]:
################## a.get_shape().ndims
import tensorflow as tf
import numpy as np

sess = tf.InteractiveSession()
a = np.arange(27).reshape([3,3,3])
a = tf.convert_to_tensor(a)
print(a.eval())
print(a.get_shape().ndims)

[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]

 [[18 19 20]
  [21 22 23]
  [24 25 26]]]
3


In [12]:
################### axis
import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()
img = tf.Variable(tf.convert_to_tensor(np.arange(16).reshape([2,2,2,2])))
axis = list(range(len(img.get_shape()) - 1))
print(axis)
mean, variance = tf.nn.moments(img, axis)
tf.global_variables_initializer().run()

print(img.eval())
print('------3--------')
print(sess.run(tf.reduce_sum(img,3)))
print('------2--------')
print(sess.run(tf.reduce_sum(img,2)))
print('------1--------')
print(sess.run(tf.reduce_sum(img,1)))
print('------0--------')
print(sess.run(tf.reduce_sum(img,0)))


[0, 1, 2]
[[[[ 0  1]
   [ 2  3]]

  [[ 4  5]
   [ 6  7]]]


 [[[ 8  9]
   [10 11]]

  [[12 13]
   [14 15]]]]
------3--------
[[[ 1  5]
  [ 9 13]]

 [[17 21]
  [25 29]]]
------2--------
[[[ 2  4]
  [10 12]]

 [[18 20]
  [26 28]]]
------1--------
[[[ 4  6]
  [ 8 10]]

 [[20 22]
  [24 26]]]
------0--------
[[[ 8 10]
  [12 14]]

 [[16 18]
  [20 22]]]


In [6]:
################ tf.nn.moments
import tensorflow as tf

img = tf.Variable(tf.random_normal([2,2,4]))
axis = list(range(len(img.get_shape()) - 1))
mean, variance = tf.nn.moments(img, axis)
print(axis)
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

print(img.eval())
print('------mean-----------')
print(mean.get_shape())
print(mean.eval())
print('------vari-----------')
print(variance.get_shape())
print(variance.eval())

[0, 1]
[[[-0.51745361  0.78703845 -0.28692922 -0.2645447 ]
  [-0.27363697  1.9616046  -0.07952929 -0.20902185]]

 [[ 2.51147103 -2.80260324  0.03841113  1.52014184]
  [-0.71927398  1.45612586  1.07170784 -0.95238084]]]
------mean-----------
(4,)
[ 0.25027663  0.35054141  0.18591511  0.02354862]
------vari-----------
(4,)
[ 1.72923088  3.48767304  0.27510718  0.83232915]


In [8]:
#-0.51745361-0.27363697+2.51147103-0.71927398
1.0011064699999999/4

0.25027661749999996

In [1]:
################## tf.nn.moments
import tensorflow as tf

img = tf.Variable(tf.random_normal([128,32,32,64]))
axis = list(range(len(img.get_shape()) - 1))
mean, variance = tf.nn.moments(img, axis)
print(axis)
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
print(mean.eval())
print()
print(variance.eval())
print(variance.get_shape())

[0, 1, 2]
[ -8.67179711e-04  -2.37519387e-03  -3.43308086e-03  -2.99280393e-03
   2.66283314e-05  -9.49081557e-04   3.88997258e-03   2.60121142e-03
   9.87584121e-04  -6.85347302e-04   2.63960217e-03  -3.50445713e-04
   9.36817261e-04   1.72128517e-03  -2.12114374e-03  -1.27304392e-03
  -1.10820401e-03  -4.19131899e-03  -1.93832475e-05  -1.29306762e-04
  -5.57611790e-03  -3.42080428e-04   1.92669639e-03  -3.96887248e-04
   5.20647923e-03  -7.20892742e-04   3.14607983e-03   2.36615026e-03
  -8.03037663e-04  -2.72341282e-03   1.91440675e-04  -2.44098855e-03
   5.47786942e-04  -9.43349907e-04  -1.41827588e-03  -3.64326173e-04
   7.30846950e-04  -4.51498805e-03   3.13573866e-03   1.80195807e-03
  -6.79371820e-04   7.76889327e-04   3.83072719e-03   1.82744092e-03
  -1.25084957e-03   5.90366661e-04  -2.00457507e-04   2.89174123e-03
   2.98392819e-03   1.26337260e-03   1.44310307e-03  -3.27450084e-03
  -6.76838390e-04  -4.37516673e-03  -1.26826938e-03  -2.21414003e-03
  -3.63026047e-03   2.34